In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
from datetime import datetime
from loguru import logger
import numpy as np
import spatialdata as sd
from napari_spatialdata import Interactive

from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.object_quantification.controller import QuantificationController

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [ ]:
# load analysis configuration
settings_path = r'C:\BLCA-1_Analysis\analysis_settings_BLCA1.yaml'

settings = load_analysis_settings(settings_path)
settings

### Define the logger

In [ ]:
log_file = settings['log_dir'] / f"cores_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
logger.add(lambda msg: print(msg, end=""))
logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for the analysis

In [ ]:
core_dir = Path(settings['analysis_dir']) / 'cores'
path_list = [core_dir / f for f in os.listdir(core_dir)]
path_list.sort()
path_list

### Setup

In [ ]:
controller = QuantificationController(
    mask_keys={'cell': 'instanseg_cell', 'nucleus': 'instanseg_nucleus'},
    table_name = 'instanseg_data',
    cytoplasm_components=('instanseg_cell', 'instanseg_nucleus'),
    cytoplasm_mask_name='cyto'
)

### Quantify

In [ ]:
for sd_path in path_list[:1]:
    
    # load data
    logger.info(f'Processing {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    controller.run(sdata)

In [ ]:
# view anndata

sd_path = r'C:\BLCA-1_Analysis\cores\Core_006.zarr'
sdata = sd.read_zarr(sd_path)
sdata

In [4]:
sdata

SpatialData object, with associated Zarr store: C:\BLCA-1_Analysis\cores\Core_006.zarr
├── Images
│     ├── '53BP1': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD3': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD8a': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11C': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD11b': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD20': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD31': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD44': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD45RO': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD68': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD127': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CD163': DataTree[cyx] (1, 7168, 7168), (1, 3584, 3584)
│     ├── 'CDC6': DataTree[cyx] (1, 7168, 7168), (1